In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy
import re
import time
print ("******* Wellcome to BestShoes ********")

******* Wellcome to BestShoes ********


In [2]:
#--->Load Html Document from www.Asos.com
def load_soup_object(html_Doc):
    soup_obj = BeautifulSoup(html_Doc.text ,'html.parser')
    return soup_obj

In [3]:
def ExtractInfo_FromSite(link, respone, agent, numberCatalog , Gender, IsfirstColor):
    Shoes_Item = load_soup_object(respone)      #BeautifulSoup
    
    ProductKey , Price = "" , 0
    Name , Color , Commpersion , Type , Brand = "" , "" , "" , "" , ""
    Reviews, GlobalRank , RankSize , RankComfort ,RankQuality = 0 ,0,0,0 ,0
    MaterialSoleRubber , MaterialSolePolyester , MaterialSoleTextile ,MaterialSoleThermoplasticPolyurethane , MaterialSoleOther, MaterialSoleLeather= 0,0,0,0,0,0
    MaterialUpperRubber , MaterialUpperPolyester , MaterialUpperTextile , MaterialUpperThermoplasticPolyurethane, MaterialUpperOther , MaterialUpperLeather =0, 0,0,0,0,0
    templink = []
    
    
    driver = webdriver.Chrome("C:\Program Files (x86)\Google\chromedriver.exe")   #->>>Seleinum
    driver.implicitly_wait(10)
    driver.get(link)
    try:
        GlobalRank_Item = WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,'.FPW6c .X3LJZ .I21qs')))
    except:
        GlobalRank_Item = []
    Name_Item = driver.find_elements(By.CSS_SELECTOR, "#pdp-react-critical-app .jcdpl")
    Price_Item = driver.find_elements(By.CSS_SELECTOR,'.MwTOW')
    Color_Item = driver.find_elements(By.CSS_SELECTOR,'.aKxaq')
    Rank_Item = driver.find_elements(By.CSS_SELECTOR,'.cnock .eJLtp')
    Reviews_Item = driver.find_elements(By.CSS_SELECTOR,".FPW6c .X3LJZ div")

    #--->Other link for other colors
    ColorList_other_ItemLinks = driver.find_elements(By.CSS_SELECTOR,"li .PSk4r")
    for x in ColorList_other_ItemLinks:
        templink.append(x.get_attribute('href'))
    for name in Name_Item:
        Name = name.text
    for color in Color_Item:
        Color=color.text 
    for price in Price_Item:
        if price.text.find(' ')>=0:
            if price.text.count(' ') == 1:
                Price=price.text[:price.text.index(' ')]
            else:
                Price=price.text[price.text.find(' ')+1:price.text.rindex(' ')]
    for rank in GlobalRank_Item:
        GlobalRank = rank.text
    if len(Reviews_Item) != 0:
        Reviews = Reviews_Item[3].text[1:-1]
    i=0
    for rank in Rank_Item:
        style = rank.value_of_css_property('left')
        left_value = style.replace("px", "")
        if i==0:
            RankSize = left_value
        if i ==1:
            RankComfort = left_value
        if i ==2:
            RankQuality = left_value
        i+=1
    driver.close()
    
    #Using With BeaotifulSoup
    Productkey_Item = Shoes_Item.find_all("p", class_="Jk9Oz") 
    Type_Item = Shoes_Item.select("#productDescriptionDetails .F_yfF")
    Brand_Item = Shoes_Item.select(".accordion-item-module_contentWrapper__h3KoR .F_yfF a")
    Details_Item = Shoes_Item.select("#productDescriptionDetails .F_yfF li")
    Compersion = Shoes_Item.select("#productDescriptionAboutMe .accordion-item-module_content__2cDKX .F_yfF")
    upper, sole = "" , ""
    for com in Compersion:
        compersion = com.get_text().replace(' ', '') #delete space
        if compersion.find("Sole:")>= 0 and compersion.find("Upper:")>= 0:
            sole = compersion[compersion.index("Sole:"):compersion.index("Upper:")]
            upper = compersion[compersion.index("Upper:"):]
        elif compersion.find("Sole:")>= 0:
            sole = compersion[compersion.index("Sole:"):]
        elif compersion.find("Upper:")>= 0:
            upper = compersion[compersion.index("Upper:"):]
        numberSole = re.findall(r'\d+', sole)
        numberUpper  =  re.findall(r'\d+', upper)
        sum=0
        for x in numberSole:
            if "Textile" == sole[sole.find(x)+len(x)+1:sole.find(x)+len(x)+7+1]:
                sole = sole[sole.find("Textile"):]
                MaterialSoleTextile = x
            elif "Rubber" == sole[sole.find(x)+len(x)+1:sole.find(x)+len(x)+6+1]:
                sole = sole[sole.find("Rubber"):]
                MaterialSoleRubber = x
            elif "Polyester" == sole[sole.find(x)+len(x)+1:sole.find(x)+len(x)+9+1]:
                sole = sole[sole.find("Polyester"):]
                MaterialSoleTextile = x
            elif "ThermoplasticPolyurethane" == sole[sole.find(x)+len(x)+1:sole.find(x)+len(x)+25+1]:
                sole = sole[sole.find("ThermoplasticPolyurethane"):]
                MaterialSoleThermoplasticPolyurethane = x
            elif "Leather" == sole[sole.find(x)+len(x)+1:sole.find(x)+len(x)+7+1]:
                sole = sole[sole.find("Leather"):]
                MaterialSoleLeather = x
            else:
                sole = sole[sole.find(x)+len(x)+1:]
                sum += int(x)
                MaterialSoleOther = x
        sum =0
        for x in numberUpper:
            if "Textile" == upper[upper.find(x)+len(x)+1:upper.find(x)+len(x)+7+1]:
                upper = upper[upper.find("Textile"):]
                MaterialUpperTextile = x
            elif "Rubber" == upper[upper.find(x)+len(x)+1:upper.find(x)+len(x)+6+1]:
                upper = upper[upper.find("Rubber"):]
                MaterialUpperRubber = x
            elif "Polyester" == upper[upper.find(x)+len(x)+1:upper.find(x)+len(x)+9+1]:
                upper = upper[upper.find("Polyester"):]
                MaterialUpperPolyester = x
            elif "ThermoplasticPolyurethane" == upper[upper.find(x)+len(x)+1:upper.find(x)+len(x)+25+1]:
                upper = upper[upper.find("ThermoplasticPolyurethane"):]
                MaterialUpperThermoplasticPolyurethane = x
            elif "Leather" == upper[upper.find(x)+len(x)+1:upper.find(x)+len(x)+7+1]:
                upper = upper[upper.find("Leather"):]
                MaterialUpperLeather = x
            else:
                upper = upper[upper.find(x)+len(x)+1:]
                sum += int(x)
                MaterialUpperOther = x   
    
    
    for product_key in Productkey_Item:
        if(len(product_key.get_text())>=0):
            ProductKey = product_key.get_text()[len('Product Code: '):]
    description =""
    for i in range(len(Details_Item)):
        if i<len(Details_Item)-1:
            description += Details_Item[i].get_text()+ " , "
        else:
            description += Details_Item[i].get_text()
    Data['Details'].append(description)
    for brand in Brand_Item:
        if Brand_Item != None:
            Brand=brand.text
    for typ in Type_Item:
        Type = typ.get_text()[:typ.get_text().index(" ")]

    #Add to Data
    Data['ProductKey'].append(ProductKey)
    Data['Price'].append(Price)
    Data['Name'].append(Name)
    Data['Color'].append(Color)
    Data['Brand'].append(Brand)
    Data['NumberCatalog'].append(numberCatalog)
    Data['Gender'].append(Gender)
    Data['Type'].append(Type)
    Data['Reviews'].append(Reviews)
    Data['GlobalRank'].append(GlobalRank)
    Data['RankSize'].append(RankSize)
    Data['RankComfort'].append(RankComfort)
    Data['RankQuality'].append(RankQuality)
    Data['MaterialSoleRubber'].append(MaterialSoleRubber)
    Data['MaterialSolePolyester'].append(MaterialSolePolyester)
    Data['MaterialSoleTextile'].append(MaterialSoleTextile)
    Data['MaterialSoleThermoplasticPolyurethane'].append(MaterialSoleThermoplasticPolyurethane)
    Data['MaterialSoleLeather'].append(MaterialSoleLeather)
    Data['MaterialSoleOther'].append(MaterialSoleOther)
    Data['MaterialUpperRubber'].append(MaterialUpperRubber)
    Data['MaterialUpperPolyester'].append(MaterialUpperPolyester)
    Data['MaterialUpperTextile'].append(MaterialUpperTextile)
    Data['MaterialUpperThermoplasticPolyurethane'].append(MaterialUpperThermoplasticPolyurethane)
    Data['MaterialUpperLeather'].append(MaterialUpperLeather)
    Data['MaterialUpperOther'].append(MaterialUpperOther)
    print("Adding Product:"+ProductKey)
    
    #----> Get All The Other Page Of Shoes Which is in diffrent Price And color
    if IsfirstColor == True:
        for url in templink:
            if url != None:  #---->The correct link is usally None
                respone = requests.get(url, headers=agent , timeout=2)
                if respone.status_code == 200:
                    ExtractInfo_FromSite(url , respone ,agent, numberCatalog ,Gender, False)
    #return True #if there is Other Color ot this type, else return false

In [4]:
#UpdateData Using Selenium
#This Function deal With the Main Menu in www.Asos.com
def UpdateData(url,NumberCatalog ,Gender, agent):
    try:
        #Create Connection to Asos Server
        respone = requests.get(url, headers=agent , timeout=2)
        if respone.status_code == 200:
            print ("Respone Is ok , The page Exist")
            
            LinkListAllProduct = []
            isNextPage = True
            NumPage = 1
            number_Catalog = NumberCatalog
            driver = webdriver.Chrome("C:\Program Files (x86)\Google\chromedriver.exe")   #->>>Seleinum
            driver.implicitly_wait(10)
            driver.get(url)
            while isNextPage:
                ItemLinkNextPage = driver.find_elements(By.XPATH, "//section[@data-auto-id="+str(NumPage)+"]//a[@class='productLink_KM4PI']")
                for link in ItemLinkNextPage:
                    LinkListAllProduct.append(link.get_attribute('href'))
                NextPage = driver.find_elements(By.CSS_SELECTOR,".loadButton_i3U2b")
                if NextPage:
                    NextPage[0].click()#Open other section of products
                    NumPage +=1
                else:
                    isNextPage = False
            driver.close()
            print("Total Product By Gender:"+str(Gender)+" "+str(len(LinkListAllProduct)))
            
            #---->Extract all the Product
            print ("Start To Extract")
            for link in LinkListAllProduct:
                respone = requests.get(link, headers=agent , timeout=2)
                if respone.status_code == 200:
                    try:
                        ExtractInfo_FromSite(link , respone ,agent, number_Catalog ,Gender, True)
                        number_Catalog+=1
                    except:
                        print("Extract Not Working")
            return number_Catalog
        else:
            print ("Response Code: {0}".format(respone.status_code))
            return number_Catalog
    
    except Exception as e:
            print("Somting goes worng")
            print(str(e))
            return -1

In [5]:
#--->Create The DataFrame
def CreateDataFrame(Data):
    df = pd.DataFrame(Data)
    print(df.info())
    return df

In [6]:
#--->Save The DataFrame in Csv Doc
def SaveToCsv(df,path):
    if not df.empty:
        df.to_csv(path)
        print("\n-->Sucssesful Save DataFrame By Crawling\n")
    else:
        print("\n-->Crawling Maybe Not Succses\n")

In [7]:
#---->#Info for crawling www.asos.com

#Dictionry Object - With List Of Data
Data = {'ProductKey': [] ,'Name':[] ,'NumberCatalog':[] ,"Gender":[] ,'Brand': [] , 'Price' : [] ,'Color' : [] , 'Details': []  , 'Type' : [] , 'MaterialSoleRubber' : [] ,'MaterialSolePolyester' : [], 'MaterialSoleTextile' : [] ,"MaterialSoleThermoplasticPolyurethane":[] , "MaterialSoleOther": [], 'MaterialUpperRubber' : [] ,'MaterialUpperPolyester' : [], 'MaterialUpperTextile' : [] ,"MaterialUpperThermoplasticPolyurethane": [], 'MaterialUpperOther':[] , 'GlobalRank' : [] , "Reviews":[] , "RankSize":[] ,"RankComfort":[],"RankQuality":[] }
csvPath = "./CSV_Document/AsosData.csv"
NumberCatalog =1 

urlWomen = "https://www.asos.com/women/shoes/cat/?cid=4172"               #--->Women url
urlMen= "https://www.asos.com/men/shoes-boots-trainers/cat/?cid=4209"     #--->Men url
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

number_Catalog = UpdateData(urlWomen ,NumberCatalog, "Women", agent)
number_Catalog = UpdateData(urlMen , number_Catalog, "Men", agent)

df = CreateDataFrame(Data)
SaveToCsv(df,csvPath)
df.info()

Respone Is ok , The page Exist


C:\Users\danie\AppData\Local\Temp\ipykernel_16560\362924880.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\Program Files (x86)\Google\chromedriver.exe")   #->>>Seleinum


Somting goes worng
Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00F6F243]
	(No symbol) [0x00EF7FD1]
	(No symbol) [0x00DED04D]
	(No symbol) [0x00DD2D7A]
	(No symbol) [0x00E3BE7B]
	(No symbol) [0x00E4C196]
	(No symbol) [0x00E38386]
	(No symbol) [0x00E1163C]
	(No symbol) [0x00E1269D]
	GetHandleVerifier [0x01209A22+2655074]
	GetHandleVerifier [0x011FCA24+2601828]
	GetHandleVerifier [0x01018C0A+619850]
	GetHandleVerifier [0x01017830+614768]
	(No symbol) [0x00F005FC]
	(No symbol) [0x00F05968]
	(No symbol) [0x00F05A55]
	(No symbol) [0x00F1051B]
	BaseThreadInitThunk [0x75FA6BD9+25]
	RtlGetFullPathName_UEx [0x776E8FD2+1218]
	RtlGetFullPathName_UEx [0x776E8F9D+1165]

Respone Is ok , The page Exist
Somting goes worng
Message: target frame detached
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=109.0.5414.120)


In [ ]:
df.head()